In [1]:
print("ok")

ok


In [2]:
%pwd

'/Users/myhome/prince/End_to_end_medicalchatbot/research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'/Users/myhome/prince/End_to_end_medicalchatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
extracted_data

[Document(metadata={'source': 'Data/Medical_book.pdf', 'page': 0}, page_content='8\nTH EDITION\n5867_FM_i-xxvi  12/01/17  12:22 PM  Page i'),
 Document(metadata={'source': 'Data/Medical_book.pdf', 'page': 1}, page_content='5867_FM_i-xxvi  12/01/17  12:22 PM  Page ii'),
 Document(metadata={'source': 'Data/Medical_book.pdf', 'page': 2}, page_content='8\nTH EDITION\nBarbara A. Gylys, MEd, CMA-A (AAMA)\nProfessor Emerita\nUniversity of T oledo\nT oledo, OH\nMary Ellen Wedding, MEd, MT(ASCP), CMA (AAMA), CPC (AAPC)\nProfessor Emerita\nUniversity of T oledo\nT oledo, OH\n5867_FM_i-xxvi  12/01/17  12:22 PM  Page iii'),
 Document(metadata={'source': 'Data/Medical_book.pdf', 'page': 3}, page_content='F. A. Davis Company\n1915 Arch Street\nPhiladelphia, PA 19103\nwww.fadavis.com\nCopyright © 2017 by F. A. Davis Company\nCopyright © 2017 by F. A. Davis Company. All rights reserved. This product is protected by copyright. No part of it may be\nreproduced, stored in a retrieval system, or transmitt

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 2940


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

/var/folders/nn/3vv9s5d557bc1lhnccljp37w0000gn/T/ipykernel_68590/2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/Users/myhome/anaconda3/envs/medibot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [18]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [20]:
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,  # your document chunks
    index_name=index_name,
    embedding=embeddings
)

In [21]:
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [22]:
docsearch

In [26]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":2})

In [27]:
retrieved_docs = retriever.invoke("What is eczema?")

In [28]:
retrieved_docs

[Document(metadata={'page': 126.0, 'source': 'Data/Medical_book.pdf'}, page_content='Disease Focus 101\nDiseases and Conditions—cont’d\nTerm Definition\necchymosis\n˘ek-˘ ı-MŌ-s˘ ıs\neczema\n˘EK-z ˘e-mă\nerythema\n˘er-˘ ı-THĒ-mă\neschar\n˘ES-kăr\nimpetigo\n˘ ım-p˘e-TĪ-gō\nkeratosis\nk˘er-ă-TŌ-s˘ ıs\nkerat: horny tissue, hard; cornea\n-osis: abnormal condition; increase \n(used primarily with blood cells)\nlentigo\nl˘en-TĪ-gō\npallor\nPĂL-or\npediculosis\np˘e-d˘ ık-ū-LŌ-s˘ ıs\npedicul: lice\n-osis: abnormal condition; increase \n(used primarily with blood cells)'),
 Document(metadata={'page': 126.0, 'source': 'Data/Medical_book.pdf'}, page_content='Skin discoloration consisting of a large, irregularly formed hemorrhagic\narea with colors changing from bluish black to greenish brown or yellow;\ncommonly called a bruise (See Fig. 5-13.)\nChronic inflammatory skin condition that is characterized by erythema,\npapules, vesicles, pustules, scales, crusts, and scabs and accompanied by\nintens

In [29]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [31]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [32]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])



Acromegaly is a disorder caused by the hypersecretion of growth hormone in adults, resulting in enlarged bones in the extremities and head. Gigantism is a similar disorder that occurs in children, causing excessive growth and height. Both conditions can be treated with radiation, medication, or surgery.


In [33]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])


System: I am not able to determine the meaning of "stats" as it is not a complete word or a medical term. Can you provide more context or information?
